# Obtain details of edge circadian score in each path

In [1]:
import numpy as np
import pandas as pd
import src.calculate_edge_circa_score as cal
import requests

In [2]:
# read in processed CircaDB data 
circa_db = pd.read_csv('data/circa_db_mapped.tsv', sep = '\t')
circa_db.head()

,gene_id,Fat SQ_fdr,Fat Visceral_fdr,Aorta_fdr,Artery Coronary_fdr,Artery Tibial_fdr,Colon_fdr,Esophagus_fdr,Heart Atrial_fdr,Liver_fdr,...,Artery Coronary_exp,Artery Tibial_exp,Colon_exp,Esophagus_exp,Heart Atrial_exp,Liver_exp,Lung_exp,Nerve Tibial_exp,Pituitary_exp,Thyroid_exp
0,653635,0.6432,0.1526,0.8443,0.7712,0.9549,0.5059,0.2928,0.6953,0.9732,...,12.300,11.590,12.7200,12.3033,5.369,5.406,13.680,19.480,15.840,19.2550
1,79854,0.7652,0.2412,0.0473,0.0002,0.6314,0.8602,0.0820,0.6481,0.6405,...,5.940,8.419,5.4407,4.6607,2.962,3.240,7.024,12.110,9.898,9.7615
2,643837,0.9075,0.0774,0.7887,0.2173,0.6017,0.3294,0.0696,0.2758,0.3438,...,9.039,11.520,7.1738,7.6350,14.350,3.674,6.016,5.872,18.600,6.5845
3,26155,0.8656,0.4570,0.5405,0.6391,0.8885,0.3902,0.6801,0.8984,0.3532,...,52.240,62.410,50.8450,62.9550,30.670,28.420,57.320,69.660,57.560,66.5800
4,339451,0.8808,0.6235,0.4360,0.4999,0.7446,0.7780,0.6238,0.6645,0.9089,...,13.510,10.800,11.6200,14.5633,4.870,5.979,15.600,9.224,21.420,15.4450


In [3]:
# read in metapath data
metapath_df = pd.read_csv('data/rephetio_significant_metapaths.tsv', header = 0, sep = '\t')
metapath_df.head()

,abbreviation,length,delta_auroc,negative_log_p
0,CtDaGaD,3,12.70%,7.2485
1,CbGbCtD,3,21.70%,6.4962
2,CbGaD,2,14.50%,6.2137
3,CrCbGaD,3,8.22%,6.0926
4,CbGeAlD,3,8.43%,5.1873


In [4]:
metapaths = list(metapath_df.abbreviation)

Use Flurbiprofen (DB00712) and rheumatoid arthritis (DOID:7148) as an example. This pair has score of 1 in liver, and 0 in all the other 12 tissues. 

In [5]:
# Return score details of Flurbiprofen ~ rheumatoid arthritis
drug = 'DB00712'
disease = 'DOID:7148'
tissues = ['Aorta','Artery Coronary','Artery Tibial','Colon','Esophagus','Fat SQ','Fat Visceral','Heart Atrial','Liver',
           'Lung','Nerve Tibial','Pituitary','Thyroid']
score = cal.detail_edge_circa_score(drug, disease, tissues, circa_db, query_metapath = metapaths)

In [6]:
# edge circadian score of drug-disease pair in 13 tissues
score['edge_circa_score']

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])

In [7]:
# number of metapaths that contain gene
score['total_meta_count']

1

In [8]:
# number of paths that contain gene in CircaDB
score['total_path_count']

12

Each element represents a path. When gene in the path is not in CircaDB, no amplitude, FDR, or path importance score will be returned 

In [9]:
# example of such path
score['score_details'][0]

{'metapath': 'CbGbCpD',
 'gene_id': [5743],
 'circa_db_amp': {},
 'circa_db_fdr': {},
 'importance_score': nan}

When gene in the path is in CircaDB, amplitude, FDR, and path importance score will be returned 

In [10]:
# example of such path
score['score_details'][1]

{'metapath': 'CbGbCpD',
 'gene_id': [5742],
 'circa_db_amp': {'Aorta': 0.0936,
  'Artery Coronary': 0.1598,
  'Artery Tibial': 0.2062,
  'Colon': 0.1835,
  'Esophagus': 0.1556,
  'Fat SQ': 0.257,
  'Fat Visceral': 0.4092,
  'Heart Atrial': 0.2363,
  'Liver': 0.4059,
  'Lung': 0.0296,
  'Nerve Tibial': 0.237,
  'Pituitary': 0.008,
  'Thyroid': 0.3475},
 'circa_db_fdr': {'Aorta': 0.5199,
  'Artery Coronary': 0.8636,
  'Artery Tibial': 0.7137,
  'Colon': 0.974,
  'Esophagus': 0.6407,
  'Fat SQ': 0.7174,
  'Fat Visceral': 0.0921,
  'Heart Atrial': 0.6006,
  'Liver': 0.0357,
  'Lung': 0.6444,
  'Nerve Tibial': 0.5069,
  'Pituitary': 0.2076,
  'Thyroid': 0.6172},
 'importance_score': 1.3730819627103723}

In [11]:
# obtain all the genes that contribute to the final score
contributing_genes = []
for i in range(0, len(score['score_details'])):
    if len(score['score_details'][i]['circa_db_amp']) > 0:
        for j in range(0, len(score['score_details'][i]['gene_id'])):
            contributing_genes.append(score['score_details'][i]['gene_id'][j])

In [12]:
contributing_genes

[5742, 5742, 5742, 5742, 5742, 5742, 5742, 5742, 5742, 5742, 5742, 5742]

All 12 paths traverse the same gene: PARVA (5742)